### Some OOP for practice (will be deleted)

In [2]:
class Downloader():
    def __init__(self):
        self.name = "Pepa"
          

In [3]:
A_person = Downloader()

In [4]:
A_person.name 

'Pepa'

# Start of a nice jupyter

In [5]:
from selenium import webdriver

In [6]:
import time

In [7]:
from math import ceil

In [8]:
from bs4 import BeautifulSoup

# How is the link structured

The website is located under domain name www.sreality.cz

Then it allows a client to search real estate under specified parameters. 

We specify: renting

type: flats

location: Prague

Then it displays 20 adverts per page

Here we break down how the link is structured

In [9]:
link_base = "https://www.sreality.cz"

In [10]:
search = "/hledani/pronajem/byty/praha"

In [11]:
pg = "?strana="

According to number of matches we can calculate the number of pages from which we have to get the links

In [12]:
matches = 7448

In [13]:
pages = ceil(matches/20)

In [14]:
pages

373

Because we understand the structure of the link we can create a list of all list that we want to visit

In [15]:
all_pages = [link_base+search+pg+str(k+1) for k in range(0, pages)]

In [16]:
all_pages[::372]

['https://www.sreality.cz/hledani/pronajem/byty/praha?strana=1',
 'https://www.sreality.cz/hledani/pronajem/byty/praha?strana=373']

Now we gathered all the links to give us the link that we will later scrape.
Sreality is a dynamic page so we will use selenium <3

## Selenium

### code for one page

We visit the webpage with selenium get the html code and find all the links that lead to our desired destination.

In [17]:
driver = webdriver.Chrome("C:\\chromedriver\\chromedriver.exe")
driver.get("https://www.sreality.cz/hledani/pronajem/byty/praha")
time.sleep(2)
res1 = driver.execute_script("return document.documentElement.outerHTML")
driver.quit()

In [18]:
soup = BeautifulSoup(res1, "lxml")

In [19]:
links = soup.find_all("a", {"class":"title"})

We check that we collected 21 links because there is one sponsored link per page.
Then we print the first link that is not sponsored

In [20]:
len(links)

21

In [21]:
links[1]['href']

'/detail/pronajem/byt/2+kk/praha-branik-novodvorska/4260773468'

## for many pages

Now this code is generalized for visiting all the pages that we previously set out to get links from

In [22]:
driver = webdriver.Chrome("C:\\chromedriver\\chromedriver.exe")
all_links = []
for k in range(2): # when ready replace 2 with variable "pages"
    driver.get(all_pages[k])
    time.sleep(1)
    res = driver.execute_script("return document.documentElement.outerHTML")
    soup = BeautifulSoup(res, "lxml")
    links = soup.find_all("a", {"class":"title"})
    for j in links[1:]:
        all_links.append(j['href'])
driver.quit()

In [23]:
len(all_links)

40

In [24]:
all_links[:5]

['/detail/pronajem/byt/2+kk/praha-branik-novodvorska/4260773468',
 '/detail/pronajem/byt/2+kk/praha-nusle-mecislavova/2212941404',
 '/detail/pronajem/byt/atypicky/praha-mala-strana-petrinska/3046194780',
 '/detail/pronajem/byt/2+kk/praha-strizkov-zaksinska/1682259548',
 '/detail/pronajem/byt/3+kk/praha-zabehlice-jahodova/1368604252']

This is just a begining of all the links we can get :D

# Now we need to decide what we want from individual sites.

In [36]:
link_explore = link_base + all_links[0]
driver = webdriver.Chrome("C:\\chromedriver\\chromedriver.exe")
driver.get(link_explore)
time.sleep(1)
gist = driver.execute_script("return document.documentElement.outerHTML")

In [37]:
info = BeautifulSoup(gist, "lxml")

## magic from now on 

In [175]:
lists = info.find_all("li")
dict = {}

dict["title"] = [info.find("span",{"itemprop":"name"}).findChild().get_text().replace("\n","").replace("\xa0"," ") ]
dict["adresa"] = [info.find("span",{"class": "location"}).get_text().replace("\n","").replace("\xa0"," ")]
for k in lists:
    try:
        column = k.find("label").get_text()
    except:
        break
    try:
        value = [k.find("strong").get_text()]
    except:
        try:
            value = [k.find("a").get_text(), k.find("span", {"class":"c-pois__distance ng-binding"}).get_text()]
        except:
            value = [k.find("span",{"class":"c-pois__poi-text ng-binding ng-scope"}).get_text(),k.find("span", {"class":"c-pois__distance ng-binding"}).get_text()]
    
    if value[0] == "\n\n\n\n\n\n\n":
        value[0] = k.find("strong").findChild("span",{"class": "icof icon-ok ng-scope"})["ng-if"].split(" ")[-1]
    
    #we clean the values a little bit
    keys = [val.replace("\n","") for val in value]
    keys_a = [ki.replace("\xa0"," ") for ki in keys]    

    #add it to the dictionary
    dict[column] = keys_a

print(dict)  

{'title': ['Pronájem bytu 2+kk 40 m²'], 'adresa': ['Novodvorská, Praha 4 - Braník'], 'Celková cena:': ['11 000 Kč za měsíc, + provize RK, bez poplatků'], 'ID zakázky:': ['21035'], 'Aktualizace:': ['Dnes'], 'Stavba:': ['Panelová'], 'Stav objektu:': ['Dobrý'], 'Vlastnictví:': ['Osobní'], 'Umístění objektu:': ['Centrum obce'], 'Podlaží:': ['7. podlaží z celkem 8'], 'Užitná plocha:': ['40m2'], 'Plocha podlahová:': ['47m2'], 'Lodžie:': ["'boolean-true'"], 'Sklep:': ["'boolean-true'"], 'Voda:': ['Dálkový vodovod'], 'Topení:': ['Ústřední plynové'], 'Plyn:': ['Plynovod'], 'Odpad:': ['Veřejná kanalizace'], 'Elektřina:': ['230V'], 'Cukrárna:': ['Cukrárna Simona', ' (71 m)'], 'Kino:': ['Modřanský biograf', ' (3149 m)'], 'Hřiště:': ['Dětské hřiště Nad Lesním divadlem', ' (141 m)'], 'Kulturní památka:': ['Rondokubistická vila v Hodkovičkách', ' (1872 m)'], 'Večerka:': ['Albert Supermarket', ' (180 m)'], 'Hospoda:': ['Karel MLEJNEK', ' (1012 m)'], 'Divadlo:': ['Tradiční loutkové divadlo Zvoneček', '

In [176]:
dict

{'title': ['Pronájem bytu 2+kk 40 m²'],
 'adresa': ['Novodvorská, Praha 4 - Braník'],
 'Celková cena:': ['11 000 Kč za měsíc, + provize RK, bez poplatků'],
 'ID zakázky:': ['21035'],
 'Aktualizace:': ['Dnes'],
 'Stavba:': ['Panelová'],
 'Stav objektu:': ['Dobrý'],
 'Vlastnictví:': ['Osobní'],
 'Umístění objektu:': ['Centrum obce'],
 'Podlaží:': ['7. podlaží z celkem 8'],
 'Užitná plocha:': ['40m2'],
 'Plocha podlahová:': ['47m2'],
 'Lodžie:': ["'boolean-true'"],
 'Sklep:': ["'boolean-true'"],
 'Voda:': ['Dálkový vodovod'],
 'Topení:': ['Ústřední plynové'],
 'Plyn:': ['Plynovod'],
 'Odpad:': ['Veřejná kanalizace'],
 'Elektřina:': ['230V'],
 'Cukrárna:': ['Cukrárna Simona', ' (71 m)'],
 'Kino:': ['Modřanský biograf', ' (3149 m)'],
 'Hřiště:': ['Dětské hřiště Nad Lesním divadlem', ' (141 m)'],
 'Kulturní památka:': ['Rondokubistická vila v Hodkovičkách', ' (1872 m)'],
 'Večerka:': ['Albert Supermarket', ' (180 m)'],
 'Hospoda:': ['Karel MLEJNEK', ' (1012 m)'],
 'Divadlo:': ['Tradiční lout

it looks so good, just saying

Plus, it should be rewriten as at least functions maybe OOP